In [5]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import openpyxl

In [6]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 100)

pandas version:  1.0.5


In [7]:
# 아메리카노만 
def find_americano(menu_str):
    menu_list = menu_str.split('|')
    for menu in menu_list:
        if "메리카노" in menu:
            return menu

In [19]:
def cafe():
    
    list = ['도봉1동','도봉2동','방학1동','방학2동','방학3동',
            '쌍문1동','쌍문2동','쌍문3동','쌍문4동','창1동','창2동','창3동','창4동','창5동']
    
    result = pd.DataFrame()
    
    for dong in list:
        base_url = 'https://map.naver.com/v5/api/search'
        params = {
        'caller':'pcweb',
        'query': '도봉구'  + dong +'카페',
        'type':'all',
        'searchCoord':'126.88415050506593;37.48030724508602',
        'page': 1,
        'displayCount': 300,
        'isPlaceRecommendationReplace':'true',
        'lang':'ko'
        }
        headers = {
            'user-agent' :  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }

        resp = requests.get(base_url,params=params,headers=headers)
        data = json.loads(resp.text)

        ############ 카페명, 도로명, 메뉴명 추출한 리스트 만들기 ###########
        
        name_menu_list = []    
        try:
            data_list = data['result']['place']['list']        
        except TypeError:
            print('해당 동에 카페가 없습니다.')
        for i in data_list:
            name_menu_list.append({
                
                'Name':i['name'],
                'Address':i['roadAddress'],
                'Menu':i['menuInfo']
            })
        
        # 메뉴명 None값 있는 행 삭제

        del_none = []
        for i in name_menu_list:
            if i['Menu']!= None:
                del_none.append(i)
            
         # 데이터 프레임 만들기

        cafe_americano_df = pd.DataFrame(del_none)

        # 아메리카노만     
        
        cafe_americano_df['Americano'] = cafe_americano_df['Menu'].apply(find_americano) 
        cafe_americano = cafe_americano_df.dropna(axis=0)

        # strip
        cafe_americano['Americano'] = cafe_americano['Americano'].apply(lambda x:x.strip())
        
        #  가격만
        cafe_americano['Americano_price'] = cafe_americano['Americano'].apply(lambda x:x.split(' ')[-1])
        
        # 변동가격 업주문의 지우기
               
        int_americano_price = cafe_americano[(cafe_americano.Americano_price.str.contains('변동가격') == False)]
        
        # Americano Price에서 ','지우기 / 정수형으로 바꾸기
        
        int_americano_price['Americano_price'] = int_americano_price['Americano_price'].str.replace(',','').astype('int')
        
        
        #'도봉구'인 카페만 출력
        
        addr_filtered = int_americano_price[int_americano_price['Address'].str.contains("도봉구")]
        
        # Menu,Americano Price1,Americano 삭제     
        
        addr_filtered.drop('Americano',axis=1,inplace=True)

        # 병합

        for i in range(len(list)):
            result = result.append(addr_filtered, ignore_index=True)

    # 중복항목 제거
    final_df =  result.drop_duplicates('Address', keep = 'first')

        # excel 저장
        # result.to_excel('도봉구카페.xlsx')
        
        
    # 인덱스 값 리셋해서 출력
    return final_df.reset_index()
        


In [20]:
cafe()

<ipython-input-19-773b70511422>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano'] = cafe_americano['Americano'].apply(lambda x:x.strip())
<ipython-input-19-773b70511422>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano_price'] = cafe_americano['Americano'].apply(lambda x:x.split(' ')[-1])
<ipython-input-19-773b70511422>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,index,Name,Address,Menu,Americano_price
0,0,카페152길,서울특별시 도봉구 도봉로152길 29-3,"아메리카노 3,000 | 오리진 치즈케익 1호 30,000 | 플레인롤케익 18,0...",3000
1,1,Cafe1253,서울특별시 도봉구 도봉로180길 6 태봉운동시설,"아메리카노 5,000",5000
2,2,도봉산커피,서울특별시 도봉구 도봉산4가길 4,"에스프레소 2,500 | 아메리카노 3,000 | 카페라떼 3,500 | 카푸치노 ...",3000
3,3,CAFE187,서울특별시 도봉구 도당로29길 65,"아메리카노 2,500 | 카페라떼 3,500 | 바닐라라떼 4,000 | 카페모카 ...",2500
4,4,이디야커피 도봉산점,서울특별시 도봉구 도봉산길 18 동서빌딩 1층,"아메리카노 3,200",3200
5,5,산과 산 사이 커피로드,서울특별시 도봉구 도봉로180가길 96,"아메리카노 2,800 | 카페라떼 3,800 | 카푸치노 3,800 | 허브티 3,...",2800
6,6,주인장이볶은커피,서울특별시 도봉구 마들로 772 1층 주인장이볶은커피,"원두 판매 5,500 | 7종 블랜딩 아메리카노 3,000 | 스모킹 라떼 3,50...",3000
7,7,이디야커피 도봉중앙점,서울특별시 도봉구 도봉로181길 18 1층,"아메리카노 2,800",2800
8,8,카페 오봉,서울특별시 도봉구 도당로27길 77 지상 1층,"아메리카노 2,800 | 카페라떼 3,800 | 바닐라라떼 3,800 | 카푸치노 ...",2800
9,9,제이네가게,서울특별시 도봉구 도봉로167길 7 행운빌라 104호,"제이의 브루잉 - HOT 4,000 | 제이의 브루잉 - ICE 4,300 | 라떼...",3000


In [17]:
a = cafe()
print(a['Americano_price'].mean())

<ipython-input-15-cf07027451e2>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano'] = cafe_americano['Americano'].apply(lambda x:x.strip())
<ipython-input-15-cf07027451e2>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano_price'] = cafe_americano['Americano'].apply(lambda x:x.split(' ')[-1])
C:\Users\zuhig\.conda\envs\study1\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

2910.4046242774566


In [18]:
len(a)

173